Ouvrir ce notebook dans :
<a href="https://colab.research.google.com/github/joanglaunes/deep_learning_image/blob/main/TP_MNIST_points_init.ipynb" target="_blank">Google Colab</a>
ou
<a href="https://rosenblatt.ens.math-info.univ-paris5.fr/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fjoanglaunes%2Fdeep_learning_image&urlpath=tree%2Fdeep_learning_image%2FTP_MNIST_points_init.ipynb&branch=main" target="_blank">Rosenblatt</a>

# TP - Réseaux de neurones pour des données nuages de points

Ce TP vise à démontrer les spécificités liées à l"'apprentissage sur des données de type nuages de points. On va tester
deux architectures simples pour la classification sur les images MNIST converties en nuages de points.

In [ ]:
import torch
import torch.nn as nn

import matplotlib.pyplot as plt

import time
import copy

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

Chargement des données: il s'agit des données MNIST classiques mais qui ont été converties en nuages de points 2D. Chaque donnée est un ensemble de 100 points dans le plan représentant le chiffre manuscrit.

In [ ]:
train_points, train_labels = torch.load("MNIST_100points_train.pt")
test_points, test_labels = torch.load("MNIST_100points_test.pt")

print(train_points.shape, test_points.shape)
print(train_labels.shape, test_labels.shape)

In [ ]:
# affichage des 5 premières données
for i in range(5):
    plt.subplot(1,5,i+1)
    plt.plot(train_points[i][:,1],train_points[i][:,0],'.')
    plt.axis("equal")
    plt.axis("off")
    plt.title(f"label:{train_labels[i]}")
plt.show()

## Approche naïve : perceptron multi-couche sur les données

__Question 1:__ Ecrire une classe qui implémente un perceptron à trois couches avec les spécificités suivantes:
- la dimension d'entrée doit correspondre à $n*d$ où $n=100$ (nombre de points) et $d=2$,
- les dimensions des features intermédiaires sont toutes égales à 64,
- la dimension de sortie doit être égale au nombre de classes (donc $10$ ici)

Quel est le nombre de paramètres de ce modèle ?

In [ ]:
class MLP(nn.Module):
    def __init__(self, dim_in, num_classes):
        ### TO DO ###

    def forward(self, x):
        # x.shape = (B,dim_in)
        ### TO DO ###
        # out.shape = (B,num_classes)
        return out

On instancie le modèle:

In [ ]:
n, d = train_points.shape[1:]
num_classes = 10
model = MLP(n*d,num_classes).to(device)

Hyperparamètres et fonction pour l'optimisation:

In [ ]:
num_epochs = 5
learning_rate = 0.001
batch_size = 100
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train_model(model, criterion, optimizer, scheduler=None, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
                all_inputs = train_points
                all_labels = train_labels
            else:
                model.eval()   # Set model to evaluate mode
                all_inputs = test_points
                all_labels = test_labels
            dataset_size, n, d = all_inputs.shape
            n_batches = dataset_size // batch_size
            all_inputs = torch.reshape(all_inputs,(n_batches,batch_size,n*d))
            all_labels = torch.reshape(all_labels,(n_batches,batch_size))

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for k in range(n_batches):
                inputs = all_inputs[k,...]
                labels = all_labels[k,...]
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train' and scheduler is not None:
                scheduler.step()

            epoch_loss = running_loss / dataset_size
            epoch_acc = running_corrects.double() / dataset_size

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

On lance l'optimisation:

In [ ]:
train_model(model, criterion, optimizer, num_epochs=num_epochs);

__Question 2:__ Que remarque-t-on ? Essayer d'augmenter le nombre d'époques, la dimension des couches internes, ou le nombre de couches. Comment interpréter les résultats ?

__Question 3:__ On va à présent implementer une version simplifiée de PointNet, en gardant seulement l'idée essentielle de l'architecture. On reprend le principe d'un perceptron multicouche, mais agissant indépendamment sur chaque point du nuages de points. Ainsi:
- la dimension d'entrée doit correspondre à $d=2$,
- les dimensions des features intermédiaires sont toutes égales à 64,
- une étape de max pooling global est appliqué avant la dernière couche (i.e. on calcule le max terme à terme des $n=100$ features maps)
- la dimension de sortie doit être égale au nombre de classes (donc $10$ ici)

Quel est le nombre de paramètres de ce modèle ?

In [ ]:
class BasicPointNet(nn.Module):
    def __init__(self, n, d, num_classes):
        super(BasicPointNet, self).__init__()
        ### TO DO ###

    def forward(self, x):
        # x.shape = (B,n,2)
        ### TO DO ###
        # out.shape = (B,num_classes)
        return out

__Question 4:__ Tester l'entraînement de ce nouveau modèle (attention, ceci suppose de faire une petite modification à la fonction d'entraînement). Que remarque-t-on ?

In [ ]:
model = BasicPointNet(n,d,num_classes).to(device)
train_model(model, criterion, optimizer, num_epochs=num_epochs);